라이브러리

In [7]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive', force_remount = True)
os.chdir("/gdrive/MyDrive/AIFFEL/230710")
os.getcwd()

Mounted at /gdrive


'/gdrive/MyDrive/AIFFEL/230710'

데이터 전처리

In [5]:
data_dir = 'train.csv'
df = pd.read_csv(data_dir)

#data shuffle
df = df.sample(frac=1)

def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)
    sentence = sentence.strip()

    return sentence

def check_class(it):
    if '협박' in it:
        return 0
    elif '갈취' in it:
        return 1
    elif '직장 내 괴롭힘' in it:
        return 2
    elif '기타 괴롭힘' in it:
        return 3
    else:
        return 4

df['data'] = df['conversation'].apply(lambda it : preprocess_sentence(it))
df['label'] = df['class'].apply(lambda it: check_class(it))

df = df[['data', 'label']]

df.head(3)

,data,label
2352,야 . 우리 저번에 술 마신 날 너 동영상 찍었거든 ? 이거 너희 가족한테 보여주면...,1
1578,어이 김씨 가져왔지 ? 모으고 있는중입니다 머 ? 내 말을 입으로 쳐드셨나 ? 이 ...,0
2612,야 ! 응 ? ? 나 ? ? 그래 너 ! 으 냄새나 냄새 ? 무슨 냄새 ? 무슨 냄...,3


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(df['data'],
                                                    df['label'],
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = df['label'])

print(X_train.shape[0])
print(X_test.shape[0])

3160
790


모델학습 및 정확도 확인

In [9]:
tfidf = TfidfVectorizer(min_df = 5, ngram_range=(1, 3))
X_train_tf = tfidf.fit_transform(X_train)

model1 = LinearSVC(random_state = 0, tol = 1e-5)
model1.fit(X_train_tf, Y_train)

X_test_tf = tfidf.transform(X_test)
Y_pred = model1.predict(X_test_tf)
print("accuracy:", accuracy_score(Y_test, Y_pred))

accuracy: 0.8265822784810126
